# Language Model

## Initialization of Datasets

### Storing nGram Data

In [ ]:
import pandas as pd

tl_ngram_data = pd.read_json('src/json data/Example-Based/Language Model/dict_tl_ngram.json')
il_ngram_data = pd.read_json('src/json data/Example-Based/Language Model/dict_il_ngram.json')

### Storing in lists

In [ ]:
tl_ngram_list = tl_ngram_data['Tagalog nGram'].tolist()
il_ngram_list = il_ngram_data['Ilokano nGram'].tolist()

### Language Model Tagalog to Ilokano Dictionary

In [ ]:
dict_tl_il_lang_mod = pd.DataFrame(columns=['Tagalog Structure', 'Ilokano Structure'])

## Matcher

In [ ]:
def match_tl_il_lang_mod():
    sp_index = 0
    
    tl_struct_list = []
    il_struct_list = []
    
    for tl_ngram_sen in tl_ngram_list:
        wp_index = 0
        
        il_ngram_sen = il_ngram_list[sp_index]
        
        # print(sp_index)
        
        for tl_ngram in tl_ngram_sen:
            # print(tl_ngram)
            
            if len(tl_ngram_sen) == len(il_ngram_sen):
                curr_il_ngram = il_ngram_sen[wp_index]
                # print('current_il_ngram:', curr_il_ngram)
                
                try:
                    next_il_ngram = il_ngram_sen[wp_index + 1]
                    # print('next_il_ngram: ', next_il_ngram)
                except:
                    pass
                try:
                    if (wp_index - 1) != -1:
                        prev_il_ngram = il_ngram_sen[wp_index - 1]
                        # print('prev_il_ngram: ', prev_il_ngram)
                except:
                    pass
                
                """
                    Counter for the number of words in the nGram
                """
                
                if tl_ngram == 'SW':
                    if tl_ngram not in tl_struct_list:
                        tl_struct_list.append(tl_ngram)
                    if curr_il_ngram not in il_struct_list:
                        il_struct_list.append(curr_il_ngram)
                        
                if tl_ngram not in tl_struct_list:
                    tl_struct_list.append(tl_ngram)
                    inDict = False
                else:
                    temp_index = tl_struct_list.index(tl_ngram)
                    inDict = True
                    
                if not inDict:
                    il_struct_list.append(curr_il_ngram)
                        
            
            wp_index += 1
        sp_index += 1
        
    print(tl_struct_list)
            

In [ ]:
match_tl_il_lang_mod()

COUNTER

In [ ]:
##TAGALOG COUNTER
temp_pos_list= tl_ngram_data['Tagalog nGram']


tl_new_list2 = []
tl_new_list = []
for pos_sen in temp_pos_list:
    tl_new_list = []
    pos_index = 0
    curr_pos = 0
    next_pos = ""
    next2_pos= ""
    next3_pos= ""
    for gram in pos_sen:
            for pos in gram:
                if pos_index != curr_pos and pos_index != 0:
                    pass
                else:
                    if pos == 'VB':
                        tl_new_list.append(1)
                    if pos == 'NN':
                        tl_new_list.append(2)
                    if pos == 'JJ':
                        tl_new_list.append(3)
                    if pos == 'RB':
                        tl_new_list.append(4)
                    if pos == 'CC':
                        tl_new_list.append(0)
                    if pos == 'PR':
                        tl_new_list.append(0)
                    if pos == 'DT':
                        tl_new_list.append(0)
                    if pos == 'SW':
                        tl_new_list.append(0)

    tl_new_list2.append(tl_new_list)        
            
    pos_index += 1
    


  

##ILOKANO COUNTER

il_temp_pos_list= il_ngram_data['Ilokano nGram']


il_new_list2 = []
il_new_list = []
for pos_sen in il_temp_pos_list:
    il_new_list = []
    pos_index = 0
    curr_pos = 0
    next_pos = ""
    next2_pos= ""
    next3_pos= ""
    for gram in pos_sen:
            for pos in gram:
                if pos_index != curr_pos and pos_index != 0:
                    pass
                else:
        
                    if pos == 'VB':
                        il_new_list.append(1)
                    if pos == 'NN':
                        il_new_list.append(2)
                    if pos == 'JJ':
                        il_new_list.append(3)
                    if pos == 'RB':
                        il_new_list.append(4)
                    if pos == 'CC':
                        il_new_list.append(0)
                    if pos == 'PR':
                        il_new_list.append(0)
                    if pos == 'DT':
                        il_new_list.append(0)
                    if pos == 'SW':
                        il_new_list.append(0)

    il_new_list2.append(il_new_list)
            
    pos_index += 1
    
    
print(il_new_list2) 
#print(tl_new_list2) 


 

